# 03_occurrence_period_distribution

In [ ]:
import os
import sys
import json
import pathlib
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import timedelta
import traceback

current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
with open(parent_dir.joinpath("config.json")) as file:
    cfg = json.load(file)

In [ ]:
import pymssql
driver = cfg["dbms"]
db_cfg = cfg[driver]
conn = pymssql.connect(server=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], database=db_cfg['@database'], as_dict=False)

In [ ]:
drug_order = ["Acetaminophen", "Vancomycin", "Naproxen", "Celecoxib", "Acyclovir"]

In [ ]:
def executeQuerynfetchall(conn, sql_query):
    """
        SQL 쿼리 실행 후 결과를 반환하는 함수
    """
    result = None
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql_query)
            result = cursor.fetchall()
        conn.commit()
    except:
        traceback.print_exc()
    return result

### 1. distribution of duration of abnormal

In [ ]:
def get_result_of_occurrence_period(conn, drug_name):
    SQL = """SELECT n_diff FROM {@person_database_schema}.{@target_person_table} WHERE n_diff IS NOT NULL"""
    sql_param_dict = {}
    sql_param_dict["@person_database_schema"] = db_cfg["@person_database_schema"]
    sql_param_dict["@target_person_table"] = f"person_{drug_name}"
    query = SQL.format(**sql_param_dict)
    values = executeQuerynfetchall(conn, query)
    values = [value[0] for value in values]
    return values

In [ ]:
result_dir = current_dir.joinpath("result")
result_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
occur_period_dict = {}
for drug_name in cfg["drug"].keys():
    values = get_result_of_occurrence_period(conn, drug_name)
    if len(values) >= 20:
        occur_period_dict[drug_name] = values

with open(result_dir.joinpath("occurrence_period_distribution.json"), "w") as file:
    json.dump(occur_period_dict, file)

In [ ]:
# save figures 
drug_counts = len(drug_order)
fig = plt.figure(figsize=(5*drug_counts, 2))
for i, drug_name in enumerate(drug_order):
    if drug_name not in occur_period_dict.keys():
        continue
    values = occur_period_dict[drug_name]
    ax = plt.subplot(1, drug_counts, i+1)
    sns.histplot(values, kde=False, bins=60, color="blue", alpha=0.3, label=drug_name)
    mean, std, median = np.mean(values), np.std(values), np.median(values)
    iqr_low, iqr_high = np.quantile(values, 0.25), np.quantile(values, 0.75)
    ax.axvline(median, color="red", linestyle="--", label="median")
    ax.text(0.98, 0.95, f"median: {median:.1f} day ({iqr_low:.1f}-{iqr_high:.1f})", transform=plt.gca().transAxes, fontsize=10, verticalalignment="top", horizontalalignment="right")
    ax.set_xlim(-1, 61)
    ax.set_xlabel("days")
    ax.set_ylabel("number of patients")

plt.savefig(result_dir.joinpath("occurrence_period_distribution.png"), dpi=300, bbox_inches="tight")
plt.show()